In [52]:
#### IMPORTS  ####
from pathlib import Path
from typing import Tuple, List
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import tqdm
import random
import datetime
import pathlib
import pytz
import glob
import re
from google.colab import drive
from torch.autograd import Variable
import seaborn as sns
import os
import shutil
import copy
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import scipy.io
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pd.DataFrame

In [53]:
HEIGHT = 30
WIDTH = 34

In [137]:
def matlab_to_datetime(mtime):
  # from matlab datenum to python datetime and round to the nearest minute
  ptime = [datetime.fromordinal(int(mtime[i])) + timedelta(days=mtime[i]%1) - timedelta(days = 366) for i in range(0,len(mtime))]
  ptimernd = [datetime(ptime[i].year, ptime[i].month, ptime[i].day, ptime[i].hour, ptime[i].minute)+ timedelta(minutes=(ptime[i].second>30)) for i in range(0,len(mtime))]
  ptime = np.asarray(ptimernd)
  return ptime

def read_matlab_event_file(file_name):
  # read .mat file of event struct
  data = scipy.io.loadmat(file_name=file_name, squeeze_me=True, struct_as_record=True)
  data = data['event_struc']
  event_num = data.shape[0]
  print(['Number of events: ',event_num])
  ls = []
  for en in range(0,event_num):
    peak_time = matlab_to_datetime([data[en][0]])[0]
    event_start = matlab_to_datetime([data[en][1]])[0]
    event_end = matlab_to_datetime([data[en][2]])[0]
    duration_h = data[en][3]
    flow_times = matlab_to_datetime(data[en][4])
    flow_vals = data[en][5]
    radar_rain = data[en][6]
    radar_times = matlab_to_datetime(data[en][7])
    flow_vals_interp = np.interp(data[en][7], data[en][4], data[en][5]) # TODO: 1) flow time goes beyond radar time and should be added. 2) peak is truncated due to the interpolation
    ls.append([peak_time,event_start,event_end,duration_h,flow_times, flow_vals, radar_rain, radar_times, flow_vals_interp])

  pd_events = pd.DataFrame(data=ls, columns = ['peak_time','start','end','duration_h','flow_times', 'flow_vals', 'radar_rain', 'radar_times', 'flow_vals_interp'])

  return pd_events

def reshape_data(x: np.ndarray, y: np.ndarray, seq_length: int) -> Tuple[np.ndarray, np.ndarray]:
  """
  Reshape matrix data into sample shape for LSTM training.
  :param x: Matrix containing input features column wise and time steps row wise
  :param y: Matrix containing the output feature.
  :param seq_length: Length of look back days for one day of prediction 
  :return: Two np.ndarrays, the first of shape (samples, length of sequence,
    number of features), containing the input data for the LSTM. The second
    of shape (samples, 1) containing the expected output for each input
    sample.
  """
  num_samples, num_features = x.shape
  x_new = np.zeros((num_samples - seq_length + 1, seq_length, num_features))
  y_new = np.zeros((num_samples - seq_length + 1, 1))
  for i in range(0, x_new.shape[0]):
    x_new[i, :, :num_features] = x[i:i + seq_length, :]
    y_new[i, :] = y[i + seq_length - 1, 0]
  return x_new, y_new

In [126]:
PATH_ROOT = "drive/MyDrive/Efrat/"# Change only here the path
file_name=PATH_ROOT+'/Data/Israel/event_struc.mat'
pd_events = read_matlab_event_file(file_name)

['Number of events: ', 22]


In [138]:
class Israel(Dataset):
  """
  Torch Dataset for basic use of data from the data set. 
  This data set provides meteorological observations and discharge of a given basin from the Israel data set.
  """
  def __init__(self, df_all_data: pd.DataFrame, event_list: List, seq_length: int, period: str=None, min_value: np.array=None, max_value: np.array=None, lead=0):
    """Initialize Dataset containing the data of a single basin.
    :param seq_length: Length of the time window of meteorological input provided for one time step of prediction.
    :param period: (optional) One of ['train', 'eval']. None loads the entire time series.
    :param dates: (optional) List ofthe start and end date of the discharge period that is used.
    """
    self.seq_length = seq_length
    self.event_list = event_list
    self.period = period
    self.min_value = min_value
    self.max_value = max_value
    self.mean_y = None
    self.std_y = None
    self.lead = lead
    self.num_features = None
    self.event_length_list = []
    # load data
    self.x , self.y= self._load_data(df_all_data)
    # store number of samples as class attribute
    self.num_samples = self.x.shape[0]
    # store number of features as class attribute

  def __len__(self):
    return self.num_samples

  def __getitem__(self, idx: int):
    return self.x[idx], self.y[idx]

  def _load_data(self, df_all_data):
    """Load input and output data from dataframe.    """
    first_event = True
    for event_idx in self.event_list:
      event_size = len(df_all_data['flow_vals_interp'][event_idx])
      x_event, y_event = self._load_event(df_all_data['radar_rain'][event_idx], df_all_data['flow_vals_interp'][event_idx], event_size)
      print(f"Event {event_idx}, x.shape = {x_event.shape}, y.shape = {y_event.shape}")
      self.event_length_list.append(len(y_event))
      if first_event:
        x, y = x_event, y_event
        first_event = False
      else:
        x = np.concatenate([x, x_event], axis=0)
        y = np.concatenate([y, y_event])
    if self.period == 'train':
      self.min_value  = x.min()
      self.max_value = x.max()
    # Normalizing the rain data  
    x -= self.min_value
    x /= (self.max_value - self.min_value)
    
    if self.period == "train":
      # normalize discharge
      self.mean_y = y.mean()
      self.std_y = y.std()
      y = self._local_normalization(y, variable='output')
    print("Data set for {0} for Events: {1}".format(self.period, self.event_list))
    print("Data size for LSTM should be: (num_samples, sequence_len, num_features) = {0}".format(x.shape))    
    # convert arrays to torch tensors
    x = torch.from_numpy(x.astype(np.float32))
    y = torch.from_numpy(y.astype(np.float32))
    return x, y
  
  def _load_event(self, event_data, event_label, event_size):
    
    # Clean nan 
    event_data[np.isnan(event_data)] = 0.0
    # Reshahpe data to 1D
    x = np.reshape(event_data, (HEIGHT*WIDTH, event_data.shape[2])).T
    return reshape_data(x[:(event_size - self.lead),:], np.matrix(event_label[self.lead:]).T, self.seq_length)
  
  def _local_normalization(self, feature: np.ndarray, variable: str) -> np.ndarray:
    """Normalize input/output features with local mean/std."""
    if variable == 'output':
      feature = (feature - self.mean_y) /self.std_y
    else:
      raise RuntimeError(f"Unknown variable type {variable}")
    return feature
  
  def local_rescale(self, feature: np.ndarray, variable: str) -> np.ndarray:
    """Rescale output features with local mean/std.
      :param feature: Numpy array containing the feature(s) as matrix.
      param variable: Either 'inputs' or 'output' showing which feature will
      be normalized
    :return: array containing the normalized feature
    """
    if variable == 'output':
      feature = feature * self.std_y + self.mean_y
    else:
      raise RuntimeError(f"Unknown variable type {variable}")
    return feature

  def get_min(self):
    return self.min_value

  def get_max(self):
    return self.max_value

  def get_mean_y(self):
    return self.mean_y

  def get_std_y(self):
    return self.std_y
    
  def get_num_features(self):
    return self.num_features





In [139]:
ds_temp  = Israel(pd_events, event_list=[1,3,5,7,8], seq_length=36, period='train', lead=1)

Event 1, x.shape = (255, 36, 1020), y.shape = (255, 1)
Event 3, x.shape = (255, 36, 1020), y.shape = (255, 1)
Event 5, x.shape = (255, 36, 1020), y.shape = (255, 1)
Event 7, x.shape = (118, 36, 1020), y.shape = (118, 1)
Event 8, x.shape = (399, 36, 1020), y.shape = (399, 1)
Data set for train for Events: [1, 3, 5, 7, 8]
Data size for LSTM should be: (num_samples, sequence_len, num_features) = (1282, 36, 1020)


In [152]:
class DNN(nn.Module):
  def __init__(self, input_size: int, num_hidden_layers: int, num_hidden_units: int, dropout_rate: float=0.0):
    super(DNN, self).__init__()
    self.input_size = input_size
    self.num_hidden_layers = num_hidden_layers
    self.num_hidden_units = num_hidden_units
    self.dropout_rate = dropout_rate    
    self.input_layer = nn.Linear(self.input_size, self.num_hidden_units)
    self.hidden_layer = nn.Linear(self.num_hidden_units, self.num_hidden_units)
    self.output_layer = nn.Linear(self.num_hidden_units, 1)
    self.dropout = nn.Dropout(p=self.dropout_rate)

  def forward(self, x):
    batch_size, timesteps, ts_size = x.size()
    x = x.view(batch_size, timesteps*ts_size)
    x = self.input_layer(x)
    for i in range(0,self.num_hidden_layers):
      x = self.hidden_layer(F.relu(self.hidden_layer(x)))
    pred = self.dropout(self.output_layer(x))
    return pred


class CNN(nn.Module):
  def __init__(self, num_channels: int, input_size: int):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(num_channels, 16, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(16, 32, 3)
    self.fc1 = nn.Linear(2688, 120)
    self.fc2 = nn.Linear(120, input_size)
    self.dropout1 = nn.Dropout()

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 2688)
    x = self.dropout1(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x


class CNNLSTM(nn.Module):
#  def __init__(self, input_size: int , hidden_size: int, num_channels: int, dropout_rate: float=0.0, num_layers: int=1, num_attributes: int=1):
  def __init__(self, input_size: int , hidden_size: int, num_channels: int, dropout_rate: float=0.0, num_layers: int=1, num_attributes: int=0):
    """Initialize model    
       :param hidden_size: Number of hidden units/LSTM cells
      :param dropout_rate: Dropout rate of the last fully connected layer. Default 0.0
    """
    super(CNNLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.dropout_rate = dropout_rate    
    self.num_channels = num_channels
    self.cnn = CNN(num_channels=num_channels, input_size=(input_size - num_attributes))
    # create required layer
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_size, num_layers=num_layers, bias=True, batch_first=True)
    self.dropout = nn.Dropout(p=self.dropout_rate)
    self.fc = nn.Linear(in_features=self.hidden_size, out_features=1)
        
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    """Forward pass through the Network.  
      param x: Tensor of shape [batch size, seq length, num features] containing the input data for the LSTM network.
      :return: Tensor containing the network predictions
    """
    #print(x.size())
    batch_size, timesteps, _ = x.size()
    # cropping the "image" part of the input
    image = x[:,:, :self.num_channels*HEIGHT*W_LON]
    image= image.view(batch_size, timesteps, self.num_channels, HEIGHT*WIDTH)
    image= image.view(batch_size, timesteps, self.num_channels, HEIGHT, WIDTH)
    c_in = image.view(batch_size * timesteps, self.num_channels, HEIGHT, WIDTH)
    # CNN part
    c_out = self.cnn(c_in)
    # CNN output should in the size of input size - atrributes_size
    cnn_out = c_out.view(batch_size, timesteps, -1)
    # cropping the "image" part of the input 
    a_in = x[:,:, self.num_channels*HEIGHT*WIDTH:]
    r_in = torch.cat((cnn_out, a_in), 2)
    output, (h_n, c_n) = self.lstm(r_in)
    # perform prediction only at the end of the input sequence
    pred = self.fc(self.dropout(h_n[-1,:,:]))
    return pred
